# Imporant

In [ ]:
!pip install fuzzywuzzy
!pip install transformers
!pip install PyPDF2
!pip install textract
import json

# data manipulation
import pandas as pd
import string
# normalize nested JSON files
from pandas.io.json import json_normalize
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import PyPDF2, urllib.request , nltk , textract
from io import BytesIO
import json
#import weasyprint
import pandas as pd
from pandas.io.json import json_normalize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
from fuzzywuzzy import fuzz
import json
import logging
import multiprocessing as mp
import numpy as np
import pandas as pd
import os
import re
import regex
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import preprocessing
from nltk import SnowballStemmer
import string
import nltk
import os
import json
from nltk.corpus import stopwords
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 19.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 57.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 KB 9.9 MB/s eta 0:00:00
  

/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 84.6 MB/s eta 0:00:00


In [ ]:
import gensim
from gensim.models import Word2Vec,KeyedVectors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Process data

## Load data

In [ ]:
JSON_FEATURE_DIR = "/content/drive/MyDrive/RAjobs/try"

In [ ]:
def content(jsonfile):
    curr_id = jsonfile.split('.')[0]
    with open(os.path.join(JSON_FEATURE_DIR, jsonfile)) as f:
        raw_json = json.load(f)

        ## for new data
        raw_json = eval("["+raw_json['text']+"]")  ## convert str to dict
        raw_json = [{'section_title': x['header'], 'text': x['text']} for x in raw_json]  ## clean data
        ## for new data

        raw_text = list(map(lambda x: ' '.join(x.values()), raw_json))
        raw_text = ' '.join(raw_text)
        raw_text = ' '.join(raw_text.split())
    sentences = '\n'.join(sent_tokenize(raw_text))
    return curr_id, sentences

In [ ]:
with mp.Pool(4) as pooler:
    content_dicts = list(pooler.map(content, os.listdir(JSON_FEATURE_DIR)))

In [ ]:
content_df = pd.DataFrame(content_dicts)
content_df = content_df.rename(columns={content_df.columns[0]: "ID",content_df.columns[1]: "content"})
content_list = content_df['content'].values.tolist()

## Clean text

In [ ]:
preprocessed_text = content_df['content'].apply(gensim.utils.simple_preprocess)
preprocessed_text[0:10]

0    [introduction, header, summer, and, autumn, ar...
1    [introduction, header, osu, extensions, educat...
2    [introduction, header, potassium, is, an, esse...
3    [introduction, header, the, most, common, plan...
4    [introduction, header, new, report, from, oreg...
5    [introduction, header, we, are, continually, d...
6    [introduction, header, poisonous, plants, are,...
7    [introduction, header, corvallis, ore, new, re...
8    [introduction, header, do, you, have, gardenin...
9    [introduction, header, the, second, video, in,...
Name: content, dtype: object

In [ ]:
#model_cbow = Word2Vec(sentences=preprocessed_text, sg=0, min_count=10, workers=4, window =3, epochs = 20)

In [ ]:
def clean_text(text, tokenizer, stopwords):
    """Pre-process text and generate tokens

    Args:
        text: Text to tokenize.

    Returns:
        Tokenized text.
    """
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [ ]:
# Create text column based on title, description, and content
custom_stopwords = set(stopwords.words("english") + ["oku", "osu", "oklahoma","oregon","use","process","number","rate","per","state","percent","may","formula","figure","use","problem","program","research","need","year","include","require","member","manag","make","help","time","also","one"])
content_df["tokens"] = content_df["content"].map(lambda x: clean_text(x, word_tokenize, custom_stopwords))


In [ ]:
# docs = content_df["content"].values
# tokenized_docs = content_df["tokens"].values

In [ ]:
import nltk
nltk.download('wordnet')
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
def stem_words(df):
    lemm = nltk.stem.WordNetLemmatizer()
    content_df['lemmatized_text'] = list(map(lambda sentence:
                                     list(map(lemm.lemmatize, sentence)),
                                     content_df.tokens))

    p_stemmer = nltk.stem.porter.PorterStemmer()
    content_df['stemmed_text'] = list(map(lambda sentence:
                                  list(map(p_stemmer.stem, sentence)),
                                  content_df.lemmatized_text))
stem_words(content_df)

## Build dictionary and BoW

In [ ]:
dictionary = Dictionary(documents=content_df.stemmed_text.values)


print("Found {} words.".format(len(dictionary.values())))

Found 21887 words.


In [ ]:
def document_to_bow(df):
    df['bow'] = list(map(lambda doc: dictionary.doc2bow(doc), df.stemmed_text))

document_to_bow(content_df)

In [ ]:
content_df.head()

,ID,content,tokens,lemmatized_text,stemmed_text,bow,topic,topic_words,words
0,2022-08-22-2085-OSU,Introduction-w/o-header Summer and autumn are ...,"[introduction, wo, header, summer, autumn, sea...","[introduction, wo, header, summer, autumn, sea...","[introduct, wo, header, summer, autumn, season...","[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 3...",12,"[plant, seed, soil, root, leaf, crop, control,...","plant, seed, soil, root, leaf, crop, control, ..."
1,2022-07-01-2316-OSU,Introduction-w/o-header OSU Extensions educati...,"[introduction, wo, header, extensions, educati...","[introduction, wo, header, extension, educatio...","[introduct, wo, header, extens, educ, program,...","[(7, 1), (29, 1), (44, 1), (53, 1), (69, 3), (...",40,"[extens, commun, youth, educ, counti, learn, h...","extens, commun, youth, educ, counti, learn, he..."
2,2022-08-22-2082-OSU,Introduction-w/o-header Potassium (K) is an es...,"[introduction, wo, header, potassium, essentia...","[introduction, wo, header, potassium, essentia...","[introduct, wo, header, potassium, essenti, nu...","[(13, 6), (20, 2), (51, 9), (53, 1), (66, 1), ...",12,"[plant, seed, soil, root, leaf, crop, control,...","plant, seed, soil, root, leaf, crop, control, ..."
3,2019-05-17-2217-OSU,Introduction-w/o-header The most common plant ...,"[introduction, wo, header, common, plant, comm...","[introduction, wo, header, common, plant, comm...","[introduct, wo, header, common, plant, commun,...","[(1, 1), (7, 1), (8, 2), (13, 3), (19, 6), (40...",20,"[tree, plant, grow, soil, water, root, year, p...","tree, plant, grow, soil, water, root, year, pr..."
4,2022-02-22-2101-OSU,Introduction-w/o-header A new report from Oreg...,"[introduction, wo, header, new, report, univer...","[introduction, wo, header, new, report, univer...","[introduct, wo, header, new, report, univers, ...","[(18, 1), (26, 1), (53, 1), (69, 3), (71, 1), ...",49,"[food, farm, commun, school, local, extens, pr...","food, farm, commun, school, local, extens, pro..."


In [ ]:
def lda_preprocessing(df):
    """ All the preprocessing steps for LDA are combined in this function.
    All mutations are done on the dataframe itself. So this function returns
    nothing.
    """
    stem_words(df)
    document_to_bow(df)


In [ ]:
corpus = content_df.bow

## LDA

In [ ]:
%%time
num_topics = 20
#A multicore approach to decrease training time
LDAmodel = LdaMulticore(corpus=corpus,
                        id2word=dictionary,
                        num_topics=num_topics,
                        workers=4,
                        chunksize=4000,
                        passes=7,
                        alpha='asymmetric')

CPU times: user 1min 7s, sys: 1min 54s, total: 3min 1s
Wall time: 56.9 s


In [ ]:
LDAmodel.get_document_topics(content_df.bow.iloc[0])

[(4, 0.02129859), (6, 0.113923185), (18, 0.8397959), (19, 0.02356957)]

In [ ]:
sorted(LDAmodel.get_document_topics(content_df.bow.iloc[0]), key=lambda x: x[1])[-1][0]

18

In [ ]:
#def document_to_lda_features(lda_model, document):
    #""" Transforms a bag of words document to features.
    #It returns the proportion of how much each topic was
    #present in the document.
    #"""
    #topic_importances = LDAmodel.get_document_topics(document, minimum_probability=0)
    #topic_importances = np.array(topic_importances)
    #return topic_importances[:,1]

#content_df['lda_features'] = list(map(lambda doc:
                                      #document_to_lda_features(LDAmodel, doc),
                                      #content_df.bow))

In [ ]:
def document_to_topic(lda_model, document):
    return sorted(LDAmodel.get_document_topics(document), key=lambda x: x[1])[-1][0]

content_df['topic'] = list(map(lambda doc:
                                      document_to_topic(LDAmodel, doc),
                                      content_df.bow))

In [ ]:
content_df['topic_words'] = [[x[0] for x in LDAmodel.show_topic(y)] for y in content_df['topic']]


In [ ]:
import ast

In [ ]:
# convert you string of a list of to an actual list
#content_df['words'] = content_df['topic_words'].apply(ast.literal_eval)

# use a lambda expression with join to keep the text inside the list
content_df['words'] = content_df['topic_words'].apply(lambda x: ', '.join(x))

In [ ]:
content_df.head()

,ID,content,tokens,lemmatized_text,stemmed_text,bow,topic,topic_words,words
0,2022-08-22-2085-OSU,Introduction-w/o-header Summer and autumn are ...,"[introduction, wo, header, summer, autumn, sea...","[introduction, wo, header, summer, autumn, sea...","[introduct, wo, header, summer, autumn, season...","[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 3...",18,"[plant, fruit, seed, grow, tomato, varieti, se...","plant, fruit, seed, grow, tomato, varieti, sea..."
1,2022-07-01-2316-OSU,Introduction-w/o-header OSU Extensions educati...,"[introduction, wo, header, extensions, educati...","[introduction, wo, header, extension, educatio...","[introduct, wo, header, extens, educ, program,...","[(7, 1), (29, 1), (44, 1), (53, 1), (69, 3), (...",10,"[food, commun, counti, extens, youth, school, ...","food, commun, counti, extens, youth, school, e..."
2,2022-08-22-2082-OSU,Introduction-w/o-header Potassium (K) is an es...,"[introduction, wo, header, potassium, essentia...","[introduction, wo, header, potassium, essentia...","[introduct, wo, header, potassium, essenti, nu...","[(13, 6), (20, 2), (51, 9), (53, 1), (66, 1), ...",5,"[soil, fertil, plant, compost, test, nutrient,...","soil, fertil, plant, compost, test, nutrient, ..."
3,2019-05-17-2217-OSU,Introduction-w/o-header The most common plant ...,"[introduction, wo, header, common, plant, comm...","[introduction, wo, header, common, plant, comm...","[introduct, wo, header, common, plant, commun,...","[(1, 1), (7, 1), (8, 2), (13, 3), (19, 6), (40...",3,"[tree, plant, leaf, flower, grow, root, branch...","tree, plant, leaf, flower, grow, root, branch,..."
4,2022-02-22-2101-OSU,Introduction-w/o-header A new report from Oreg...,"[introduction, wo, header, new, report, univer...","[introduction, wo, header, new, report, univer...","[introduct, wo, header, new, report, univers, ...","[(18, 1), (26, 1), (53, 1), (69, 3), (71, 1), ...",10,"[food, commun, counti, extens, youth, school, ...","food, commun, counti, extens, youth, school, e..."


In [ ]:
content_df.to_excel('/content/drive/MyDrive/RAjobs/textanalysis_OSU_v2.xlsx',index= False)

In [ ]:
topic_group = content_df.groupby('words')

#topic_group.get_group(['speci', 'area', 'nest', 'habitat', 'control', 'wildlif', 'feed', 'anim', 'fli', 'use'])

In [ ]:
df2 = pd.DataFrame(content_df.groupby('words').count())

In [ ]:
df2['words'] = df2.index

In [ ]:
df2

,ID,content,tokens,lemmatized_text,stemmed_text,bow,topic,topic_words,words
words,,,,,,,,,
"bee, introduct, header, wo, beekeep, honey, activ, extens, coloni, prune",178,178,178,178,178,178,178,178,"bee, introduct, header, wo, beekeep, honey, ac..."
"camp, youth, learn, counselor, inform, wine, student, day, grape, plant",77,77,77,77,77,77,77,77,"camp, youth, learn, counselor, inform, wine, s..."
"compost, soil, plant, diseas, garden, pile, materi, use, caus, seed",77,77,77,77,77,77,77,77,"compost, soil, plant, diseas, garden, pile, ma..."
"crop, food, cover, seed, product, veget, temperatur, slug, plant, use",69,69,69,69,69,69,69,69,"crop, food, cover, seed, product, veget, tempe..."
"fair, hors, counti, contest, member, project, pattern, class, anim, judg",48,48,48,48,48,48,48,48,"fair, hors, counti, contest, member, project, ..."
"farm, forest, manag, land, resourc, extens, plan, introduct, tree, work",238,238,238,238,238,238,238,238,"farm, forest, manag, land, resourc, extens, pl..."
"feed, anim, forag, pastur, hay, cattl, livestock, requir, qualiti, energi",92,92,92,92,92,92,92,92,"feed, anim, forag, pastur, hay, cattl, livesto..."
"food, commun, counti, extens, youth, school, educ, activ, learn, student",482,482,482,482,482,482,482,482,"food, commun, counti, extens, youth, school, e..."
"garden, extens, master, commun, counti, volunt, fire, expert, plant, school",386,386,386,386,386,386,386,386,"garden, extens, master, commun, counti, volunt..."


In [ ]:
df2.to_excel('/content/drive/MyDrive/RAjobs/count_OSU_v2.xlsx',index= False)